In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
print(tf.__version__)

2.4.1


# Inleiding

## Data
We hebben een synthetische datasets. Deze bestaat uit twee items (`left` en `right`), die elk een bepaalde beschrijving hebben (vijf features, samengevoegd tot één string), en een kolom `match` die aangeeft of de twee items een match zijn, of niet. De beschrijving bestaat uit technische codes en serienummers. 

## Probleem
De klant wil graag weten, wanneer twee items hetzelfde zijn, om zo schaalvoordeel te kunnen behalen met de wereldwijde inkoop. Omdat er miljoenen items zijn, moet de klant efficient kunnen zoeken: gegeven 1 item, wat zijn de mogelijke matchende items?

## Onze opdracht
Onze taak is om een proof-of-concept te leveren voor een set synthetische data: wij gaan laten zien dat wij met behulp van deep learning een item-matcher kunnen bouwen, en in dat proces een vectorembedding voor items genereren (wat je item2vec kan noemen; daarover verderop meer).

## Motivatie voor Deep Learning
Dit probleem zou, voor deze bescheiden synthethische dataset, waarschijnlijk redelijk eenvoudig kunnen worden opgelost met wat handmatige feature engineering en een simpel model. De dataset van de klant is echter veel complexer (met beschrijvingen per item van over de 4000 karaters, en miljoenen items en duizenden regels die bepalen of iets een match is). Dit is daarom niet een eindmodel, maar een proof-of-concept: we proberen intuities te krijgen over of dit werkt, en wat beter of slechter werkt.

Daarnaast kan een simpel model wellicht wel zeggen: deze twee items zijn een match. Maar om in een groep van miljoenen items te zoeken naar een match, zouden we dan voor een item ook miljoenen potentiele matches moeten testen. Wanneer we een model hebben dat een itembeschrijving omzet naar een Embedding, dan kunnen we veel sneller de omgeving van een item doorzoeken.

## Structuur van de opdracht
We gaan een siamees netwerk bouwen. Dat gaat er ruwweg zo uitzien:

<img src=https://www.pyimagesearch.com/wp-content/uploads/2020/11/keras_siamese_networks_header.png width=700/>

- We hebben *twee* inputs (in dit voorbeeld zijn dat afbeeldingen)
- Elk item gaat door een feature extractor. In dit voorbeeld is dat een ConvNet genoemd. De feature extractor is een model op zichzelf, dat als output een embedding geeft (in dit voorbeeld een encoding genoemd) van een bepaald aantal dimensies (bijvoorbeeld 16). De beide items moeten door *hetzelfde* netwerk gaan.
- Vervolgens wordt de afstand tussen die twee embeddings berekend, in dit voorbeeld een euclidian distance.
- In dit voorbeeld wordt geeindigd met een sigmoid. Wij gaan dat iets anders aanpakken.

Er zijn vijf onderdelen:

* Opdracht 1: data preprocessing. We moeten de data vectoriseren, en datagenerators maken.

* Opdracht 2: We bouwen een feature extractor.

* Opdracht 3: We gebruiken de feature extractor om een siamees netwerk te bouwen.

* Opdracht 4: Verbeter het basismodel

* Opdracht 5: bonusvragen

Opdrachten 1 t/m 4 testen de dingen die we tijdens de training hebben geoefend. Daar valt ook het nabouwen van een architectuur onder die ik je geef, ook al heb je die architectuur nog niet exact zo gezien.

Opdracht 5 is een bonusvraag, waar je extra punten mee kunt scoren.





# Opdracht 1: Data verkennen en voorbereiden



In [2]:
url = 'https://raw.githubusercontent.com/uashogeschoolutrecht/tmoi-ml-19/master/data/synthetic_data_bigvocab.csv'
file = tf.keras.utils.get_file('data.csv', url)
df = pd.read_csv(file)
df

1048576/1045352 [==============================] - 0s 0us/step


,left,right,match,rule
0,vj43__i1____i1____ixk4__cdo9__,mg0___i1____w46___l0____vj43__,True,0
1,vj43__dri57_i1____qo7___cdo9__,h1____vj43__hu68__i1____fs8___,True,0
2,ao1___e17___ds90__vj43__ui1___,xa55__vj43__i1____n1____sz2___,True,0
3,tyq0__hl3___hl3___vj43__i1____,byb9__vj43__i28___yj04__i1____,True,0
4,i1____vj43__gx81__rr02__ixk4__,ixk4__i1____h3____vj43__m59___,True,0
...,...,...,...,...
14581,k2____v72___zal81_f45___hl3___,e95___qto54_bx29__sef9__md40__,False,None
14582,zal81_jy9___v72___qo7___zr6___,tvr11_jf4___fm2___fm2___jx84__,False,None
14583,h99___po16__qto54_u4____tvr11_,w2____k2____f37___xrd55_ux87__,False,None
14584,tvr11_fs8___yf5___xto84_n1____,ux54__byb9__ab50__j33___lp3___,False,None


## Beschrijving data
Je ziet hierboven twee kolommen: `left` en `right`. Dit zijn de beschrijvingen van twee items. Er zijn vijf features, die zijn samengevoegd tot een lange string. Features zijn bijvoorbeeld: `vj43` of `i1`, dat wordt dan samengevoegd tot `vj43___i1_`. De data is al gepreprocessed, dus alles is lowercase letters of cijfers. Omdat de features van verschillende lengte zijn, is de tussenruimte opgevuld met een padding-karakter, een underscore: `_`.

De kolom `match` is een boolean, die aangeeft of iets een match is, of niet. De kolom `rule` geeft aan, welke regel gebruikt is om een match te genereren.

De dataset is nog niet geshuffeled, en ook niet gebalanceerd. Verken de data voor zover je dat nodig vindt om keuzes te maken.

We gaan in deze opdracht de datasets voorbewerken, zodat de data klaar is om aan ons model gevoerd te worden. Dat mag op verschillende manieren, zolang het eindproduct maar datagenerators zijn.


# Opdracht 1a : Vectorisatie van de tokens

Vectoriseer de input op *karkaterniveau*. In de les hebben we geexperimenteerd met vectorisatie op *woord*niveau (met `TextVectorization`) , en op *zins*niveau (met pretrained modellen). Nu willen we dus vectorisatie op *karakterniveau*: elke letter of cijfer moet worden omgezet in een integer. Ons model kan namelijk niet omgaan met letters, maar wel met cijfers. We hebben dus een mapping nodig van elk karakter naar een cijfer.

Dat kan op meerdere manieren: het boek doet het met een tensorflow Tokenizer (zie hoofdstuk 16) maar je mag het ook op een andere manier doen. Zolang het de data maar geschikt maar als input voor een model.

Als je jezelf wilt controleren, kun je een string invoeren in je tokenizer en checken of er getallen uitkomen.


## Opdracht 1b : datagenerators
Onze datagenerator moet gevectoriseerde data genereren (dus integers in plaats van letters), en dan steeds een batch van tuples `(left, right)` en een label `y`. Het model moet namelijk per keer steeds een paar ontvangen (`left` en `right`) en gaat van dat paar leren, of ze een match zijn of niet.

Als je perse het vectoriseren in je model zelf wilt stoppen, als layer, mag dat ook, maar dat hoeft niet.

Een `left` of `right` item heeft dan een dimensionaliteit van `(batchsize x sequencelength)`, terwijl een label een dimensionaliteit van `(batchsize x 1)` heeft.

Hint: Shuffle en split je dataset zoals je in een productieomgeving zou doen; zorg dus dat je geen data lekt, ook niet via jouw ogen / bewustzijn...


Als je jezelf wilt testen, dan zou een `.take(1)` van je generator als x een tuple met als shape per item in je tuple `(batch x sequence)` moeten teruggeven en als y een shape `(32 x 1)`

# Opdracht 2 : Feature extractor

We hebben bij de les over word-embeddings gezien, dat we de betekenis van woorden kunnen omzetten naar een vector. Woorden met dezelfde betekenis komen dan dichter bij elkaar te liggen in een multidimensionale ruimte (bijvoorbeeld van 16 dimensies, of 50, of 300).

Dit wordt vaak word2vec genoemd. Er zijn ook sentence-embeddings, dan kun je spreken van sentence2vec. 

Onze "taal" in deze dataset is echter geen Engels, of Nederlands, maar een technische taal van serienummers en materiaalcodes. Elk item heeft een aantal van deze codes. Wij willen een item2vec gaan bouwen: dat gaan we uiteindelijk doen met een siamees netwerk. Maar eerst gaan we een feature_extractor bouwen, die als input een item krijgt in de vorm van een reeks cijfers

- Definieer een functie, die als je hem aanroept een keras Model terugstuurt. Je model wordt nog niet ge-compile-d, dat gaan we pas doen bij het uiteindelijke siamese model. We genereren vooral een model, zodat we hetzelfde model kunnen hergebruiken voor het linker en rechter item.
- Maak de functie zo, dat je verschillende argumenten kunt meegeven aan de extractor voor de verschillende lagen. Bijvoorbeeld voor de hoeveelheid embeddings etc.
- De basisarchitectuur van je extractor zou er zo moeten uitzien:

  1. Een inputlaag
  2. Een Embedding laag
  3. Een Conv1D met een aantal filters en default kernel van 3.
  4. Een MaxPool1D met default stride 2
  5. Een Dense layer met een relu activatie

Geef je model een duidelijke naam mee (bijvoorbeeld 'extractor') zodat je hem later makkelijk uit je model kunt peuteren.


Als je jezelf wilt testen: 

genereer een feature extractor met behulp van je functie. 
Wanneer je nu uit je generator een eerste batch van `(left, right)` items haalt, dan kun je één van die items (bijvoorbeeld de `left`) aan je extractor voeren. 

De input is dan bijvoorbeeld een left item met afmeting `(batch x sequencelength)` en de extractor geeft dan als output `(batch x dimensionaliteit_Dense)`

# Opdracht 3

We gaan nu de feature extractor gebruiken in een siamese netwerk. Om dat te doen, gaan we de afstand berekenen tussen de twee vectoren. Dat kan als volgt, met de functie `tf.linalg.norm`

In [3]:
links = np.random.rand(32, 16)
rechts = np.random.rand(32, 16)
distance_ = tf.linalg.norm(links - rechts, axis=1)
distance_

<tf.Tensor: shape=(32,), dtype=float64, numpy=
array([1.30810323, 1.74697942, 1.37400658, 1.46573726, 1.1034742 ,
       1.35723074, 1.71460417, 1.30288774, 1.75965259, 1.5224766 ,
       1.6393335 , 1.41884316, 0.77631058, 1.21409383, 1.93551651,
       1.74973759, 1.87968921, 1.71995256, 1.66067274, 1.42630478,
       1.35025613, 1.79716576, 1.52016704, 1.18704161, 1.73246956,
       1.10079436, 1.9983664 , 1.16001782, 2.16236654, 1.46571079,
       1.794174  , 1.24890016])>

Als je lang doortraint, zou je als afstand een 0 kunnen krijgen. Dat kan `NaN`'s geven. Je kunt je model daartegen beveiligen door te clippen. Onderstaande voorbeeldcode zorgt dat de distance minimaal tussen 1e-14 en 100 ligt. In het voorbeeld hieronder liggen geen getallen buiten die grens; pas de waardes maar eens aan om te zien hoe dit werkt.

In [4]:
tf.clip_by_value(distance_, clip_value_min=1e-14, clip_value_max=100)

<tf.Tensor: shape=(32,), dtype=float64, numpy=
array([1.30810323, 1.74697942, 1.37400658, 1.46573726, 1.1034742 ,
       1.35723074, 1.71460417, 1.30288774, 1.75965259, 1.5224766 ,
       1.6393335 , 1.41884316, 0.77631058, 1.21409383, 1.93551651,
       1.74973759, 1.87968921, 1.71995256, 1.66067274, 1.42630478,
       1.35025613, 1.79716576, 1.52016704, 1.18704161, 1.73246956,
       1.10079436, 1.9983664 , 1.16001782, 2.16236654, 1.46571079,
       1.794174  , 1.24890016])>

Definieer nu een functie, die een siamees netwerk teruggeeft.
Je hebt hiervoor een functionele API nodig!

Zorg dat je netwerk het volgende heeft:

*   Twee inputs (een voor het linker, een voor het rechter item)
*   genereer één keer een extractor via de functie die je in opdracht 2 hebt gemaakt.
*   Process vervolgens zowel je linker als je rechter input met dezelfde extractor, zodat dezelfde gewichten worden gebruikt.
*   Bereken de distance.
*   Clip de distance.
*   Reshape de distance zodat de output de vorm `(batch x 1)` heeft.
*   Laat je functie een `Model` terugsturen met twee inputs `[left, right]` en een output `[distance]`



Maak nu een model (dat je siamese noemt) met de volgende features:
- De juiste inputshape
- De juiste vocab length voor je embedding
- Een Embedding met dimensionaliteit 16
- 8 convolutional filters
- 16 units als output van de laatste Dense layer

In [ ]:
siamese = 

Zoals je wellicht is opgevallen, stuurt ons model een distance terug. Dus als wij 32 keer een tuple `(left, right)` aan het model geven, krijgen we 32 keer een distance terug (met als shape `(batch x 1)`.

In de afbeelding in het begin van de opdracht wordt de distance door een sigmoid gehaald en zo geclassificeerd. Maar dat hoeft niet: we kunnen ook een `contrastive_loss` gebruiken. [link naar tf documentatie](https://www.tensorflow.org/addons/api_docs/python/tfa/losses/contrastive_loss)

In [ ]:
from tensorflow.keras.optimizers import Adam
import tensorflow_addons as tfa
siamese.compile(loss=[tfa.losses.contrastive_loss], optimizer=Adam()) # deze code gaat ervan uit dat je je model `siamese` hebt genoemd.

Train je model voor 20 epochs en evalueer de loss.

# Opdracht 4: verbeter

Kun je deze architectuur verbeteren? 

Beantwoord eerst de volgende vragen:

1. Wat is de impact van het vergroten of verkleinen van het aantal embeddings?
2. Wat is de impact van het vergroten of verkleinen van het aantal units in de laatste Dense layer?
3. Hoe verhouden de dimensionaliteit van de embedding en het aantal units in de laatste Dense layer zich? Bijvoorbeeld als de een veel kleiner is dan de ander?
4. Wat is de impact van het vergroten of verkleinen van het aantal filters in de Conv1D layer?


Je zou dit basismodel substantieel moeten kunnen verbeteren: de val_loss kan makkelijk een factor 10 omlaag ten opzichte van het baseline model.




1.   Antwoord
2.   Antwoord
3. Antwoord
4. Antwoord



# Opdracht 5 : Evalueer

We kunnen ons model op meerdere manieren evalueren.

Een eerste vraag is: Als we een voorspelling doen met ons model, krijgen we een hoeveelheid afstanden terug. Is het inderdaad zo, dat de paren die een match zijn een lage afstand hebben? En de paren die geen match zijn, een grote afstand hebben?

Een andere vraag is: kun je een precision-recall curve maken? Beschouw dan de distance als een threshold. 

Om dit te kunnen doen met sklearn, zul je de distance moeten aanpassen zodat hij als een threshold werkt zoals sklearn verwacht. Hiervoor moet je zowel de documentatie van de contrastive loss moeten bestuderen, als die van de precision_recall_curve van sklearn.

Je kunt ook zelf iets maken: je hebt een distance, en contrastive_loss gebruikt standaard een margin van 1: dat betekent dat matches dichter dan een afstand 1 moeten zijn. Met deze informatie kun je zelf een threshold maken om te evalueren.

# Bonus

Er zijn verschillende dingen die je als bonus kunt onderzoeken. Wat suggesties:

- Ben je in staat om de extractor uit je getrainde model te halen? Hint: kijk naar `.get_layer(naam)`, waarbij `naam` de naam van jouw extractor-laag is.


- Tensorboard heeft een [embedding projector](https://projector.tensorflow.org). Kun je visualisatie maken van de embeddings die je met de extractor kunt maken die je uit je getrainde model hebt gehaald? Een andere optie is om sklearn te gebruiken, bijvoorbeeld [tsne](https://scikit-learn.org/stable/modules/manifold.html#t-sne)

- Wellicht kun je wat meer exotische architecturen inbouwen? Bijvoorbeeld een residual unit, of een (variatie op) inception?